In [4]:
!pip install datasets

## Part 1

In [5]:
# Question 1(a): What is the size of the vocabulary formed from your training data?

from datasets import load_dataset
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Load the dataset
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train']

# Initialize an empty set to store unique words
vocabulary = set()

# Tokenize each review in the training dataset and update the vocabulary set
for text in train_dataset['text']:
    tokens = word_tokenize(text.lower())
    vocabulary.update(tokens)

# Print the size of the vocabulary
print("The size of the vocabulary is:", len(vocabulary))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

The size of the vocabulary is: 18029


In [8]:
# Question 1(b): How many OOV words exist in your training data?

import numpy as np

# Load the GloVe embeddings (make sure to download 'glove.6B.100d.txt' and place it in the working directory)
glove_vocab = set()
with open('glove.6B.100d.txt', 'r', encoding='utf8') as f:
    for line in f:
        word = line.split()[0]
        glove_vocab.add(word)

# Identify OOV words
oov_words = vocabulary - glove_vocab

# Print the number of OOV words
print("Number of OOV words in the training data:", len(oov_words))

Number of OOV words in the training data: 16048


In [10]:
# Question 1(c): Mitigating OOV Words by Initializing Random Embeddings

embedding_dim = 100  # Dimensionality of GloVe embeddings
import numpy as np

# Create mappings between words and indices
word2idx = {}
idx2word = {}
for idx, word in enumerate(vocabulary):
    word2idx[word] = idx
    idx2word[idx] = word

# Initialize the embedding matrix with random values
embedding_matrix = np.random.uniform(-0.05, 0.05, (len(vocabulary), embedding_dim))

# Load GloVe embeddings into the embedding matrix where possible
with open('glove.6B.100d.txt', 'r', encoding='utf8') as f:
    for line in f:
        values = line.split()
        glove_word = values[0]
        if glove_word in word2idx:
            vector = np.asarray(values[1:], dtype='float32')
            idx = word2idx[glove_word]
            embedding_matrix[idx] = vector

# Now, embedding_matrix contains GloVe embeddings for known words and random values for OOV words
print("Embedding matrix shape:", embedding_matrix.shape)

Embedding matrix shape: (18029, 100)


## Part 2

In [11]:
# Import necessary libraries
from datasets import load_dataset
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn

# Load the dataset
dataset = load_dataset("rotten_tomatoes")
train_raw = dataset['train']
validation_raw = dataset['validation']
test_raw = dataset['test']

# Build the vocabulary from training data
vocabulary = set()
for text in train_raw['text']:
    tokens = word_tokenize(text.lower())
    vocabulary.update(tokens)

# Add special tokens
vocabulary.add('<unk>')
vocabulary.add('<pad>')

# Create mappings between words and indices
word2idx = {word: idx for idx, word in enumerate(vocabulary)}
idx2word = {idx: word for word, idx in word2idx.items()}

# Initialize the embedding matrix
embedding_dim = 100  # Dimensionality of GloVe embeddings
vocab_size = len(word2idx)
embedding_matrix = np.random.uniform(-0.05, 0.05, (vocab_size, embedding_dim))

# Load GloVe embeddings (ensure 'glove.6B.100d.txt' is in your working directory)
glove_path = 'glove.6B.100d.txt'  # Update the path if necessary
with open(glove_path, 'r', encoding='utf8') as f:
    for line in f:
        values = line.strip().split()
        if len(values) == embedding_dim + 1:
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            if word in word2idx:
                idx = word2idx[word]
                embedding_matrix[idx] = vector

# Set the embedding for '<pad>' token to zeros
pad_idx = word2idx['<pad>']
embedding_matrix[pad_idx] = np.zeros(embedding_dim)

# Define a function to convert sentences to indices
def sentence_to_indices(sentence, word2idx):
    tokens = word_tokenize(sentence.lower())
    indices = []
    for token in tokens:
        if token in word2idx:
            indices.append(word2idx[token])
        else:
            indices.append(word2idx['<unk>'])  # Map unknown words to '<unk>'
    return indices

# Define the custom Dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, word2idx):
        self.texts = texts
        self.labels = labels
        self.word2idx = word2idx

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        indices = sentence_to_indices(text, self.word2idx)
        return torch.tensor(indices, dtype=torch.long), torch.tensor(label, dtype=torch.long)

# Create datasets for training, validation, and testing
train_dataset = SentimentDataset(train_raw['text'], train_raw['label'], word2idx)
val_dataset = SentimentDataset(validation_raw['text'], validation_raw['label'], word2idx)
test_dataset = SentimentDataset(test_raw['text'], test_raw['label'], word2idx)

# Define the collate_fn function for padding within batches
def collate_fn(batch):
    sequences = [item[0] for item in batch]
    labels = torch.tensor([item[1] for item in batch], dtype=torch.float)
    sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=word2idx['<pad>'])
    return sequences_padded, labels

# Create DataLoaders for training, validation, and testing
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Define the RNN model using pre-trained embeddings
class SentimentRNN(nn.Module):
    def __init__(self, embedding_matrix):
        super(SentimentRNN, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float32), freeze=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_size=256, batch_first=True, bidirectional=True, num_layers=2, dropout=0.5)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, (h_n, c_n) = self.lstm(embeds)

        # Concatenate the final hidden states from both directions
        out = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)

        out = self.dropout(out)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = torch.sigmoid(self.fc2(out))
        return out.squeeze()


# Instantiate the model
model = SentimentRNN(embedding_matrix)

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2, factor=0.5, verbose=True)

# Training loop with validation and early stopping
num_epochs = 30
patience = 5  # Early stopping patience
best_val_accuracy = 0
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)  # Gradient clipping
        optimizer.step()
        running_loss += loss.item() * sequences.size(0)
    epoch_loss = running_loss / len(train_dataset)

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for sequences, labels in val_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = model(sequences)
            predicted = (outputs >= 0.5).long()
            correct += (predicted == labels.long()).sum().item()
            total += labels.size(0)
    val_accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Learning rate scheduling
    scheduler.step(val_accuracy)

    # Check for improvement
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_no_improve = 0
        # Save the best model
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print('Early stopping!')
            break



# Load the best model and evaluate on the test set
model.load_state_dict(torch.load('best_model.pt'))
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for sequences, labels in test_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        outputs = model(sequences)
        predicted = (outputs >= 0.5).long()
        correct += (predicted == labels.long()).sum().item()
        total += labels.size(0)
test_accuracy = correct / total
print(f'Test Accuracy: {test_accuracy:.4f}')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/30], Loss: 0.6594, Validation Accuracy: 0.6482
Epoch [2/30], Loss: 0.5917, Validation Accuracy: 0.7242
Epoch [3/30], Loss: 0.5544, Validation Accuracy: 0.7073
Epoch [4/30], Loss: 0.5379, Validation Accuracy: 0.7233
Epoch [5/30], Loss: 0.5074, Validation Accuracy: 0.7223
Epoch [6/30], Loss: 0.4790, Validation Accuracy: 0.7542
Epoch [7/30], Loss: 0.4574, Validation Accuracy: 0.7552
Epoch [8/30], Loss: 0.4507, Validation Accuracy: 0.7467
Epoch [9/30], Loss: 0.4328, Validation Accuracy: 0.7655
Epoch [10/30], Loss: 0.4154, Validation Accuracy: 0.7617
Epoch [11/30], Loss: 0.4003, Validation Accuracy: 0.7617
Epoch [12/30], Loss: 0.3735, Validation Accuracy: 0.7570
Epoch [13/30], Loss: 0.3355, Validation Accuracy: 0.7674
Epoch [14/30], Loss: 0.3191, Validation Accuracy: 0.7777
Epoch [15/30], Loss: 0.2979, Validation Accuracy: 0.7552
Epoch [16/30], Loss: 0.2812, Validation Accuracy: 0.7674
Epoch [17/30], Loss: 0.2637, Validation Accuracy: 0.7730
Epoch [18/30], Loss: 0.2395, Validation 

<ipython-input-11-3c2b1135c285>:190: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pt'))


Test Accuracy: 0.7739


In [13]:
#3.1

In [12]:
class SentimentRNN_UpdateEmbeddings(nn.Module):
    def __init__(self, embedding_matrix):
        super(SentimentRNN_UpdateEmbeddings, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True  # Update the embeddings during training
        self.lstm = nn.LSTM(embedding_dim, hidden_size=256, batch_first=True, bidirectional=True, num_layers=2, dropout=0.5)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, (h_n, c_n) = self.lstm(embeds)

        # Concatenate the final hidden states from both directions
        out = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)

        out = self.dropout(out)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = torch.sigmoid(self.fc2(out))
        return out.squeeze()

In [14]:
# Instantiate the model
model = SentimentRNN_UpdateEmbeddings(embedding_matrix)

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2, factor=0.5, verbose=True)

# Training loop with validation and early stopping
num_epochs = 30
patience = 5  # Early stopping patience
best_val_accuracy = 0
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)  # Gradient clipping
        optimizer.step()
        running_loss += loss.item() * sequences.size(0)
    epoch_loss = running_loss / len(train_dataset)

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for sequences, labels in val_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = model(sequences)
            predicted = (outputs >= 0.5).long()
            correct += (predicted == labels.long()).sum().item()
            total += labels.size(0)
    val_accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Learning rate scheduling
    scheduler.step(val_accuracy)

    # Check for improvement
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_no_improve = 0
        # Save the best model
        torch.save(model.state_dict(), 'best_model_update_embeddings.pt')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print('Early stopping!')
            break

# Load the best model and evaluate on the test set
model.load_state_dict(torch.load('best_model_update_embeddings.pt'))
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for sequences, labels in test_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        outputs = model(sequences)
        predicted = (outputs >= 0.5).long()
        correct += (predicted == labels.long()).sum().item()
        total += labels.size(0)
test_accuracy = correct / total
print(f'Test Accuracy: {test_accuracy:.4f}')

Epoch [1/30], Loss: 0.6386, Validation Accuracy: 0.7261
Epoch [2/30], Loss: 0.4972, Validation Accuracy: 0.7523
Epoch [3/30], Loss: 0.3437, Validation Accuracy: 0.7674
Epoch [4/30], Loss: 0.2113, Validation Accuracy: 0.7617
Epoch [5/30], Loss: 0.1157, Validation Accuracy: 0.7636
Epoch [6/30], Loss: 0.0626, Validation Accuracy: 0.7692
Epoch [7/30], Loss: 0.0357, Validation Accuracy: 0.7477
Epoch [8/30], Loss: 0.0227, Validation Accuracy: 0.7523
Epoch [9/30], Loss: 0.0122, Validation Accuracy: 0.7523
Epoch [10/30], Loss: 0.0035, Validation Accuracy: 0.7495
Epoch [11/30], Loss: 0.0018, Validation Accuracy: 0.7533
Early stopping!


<ipython-input-14-41bf73be0b91>:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_update_embeddings.pt'))


Test Accuracy: 0.7974


In [ ]:
#3.2

In [15]:
# Load the GloVe embeddings
glove_vocab = set()
with open('glove.6B.100d.txt', 'r', encoding='utf8') as f:
    for line in f:
        word = line.split()[0]
        glove_vocab.add(word)

# Initialize the embedding matrix with random values
embedding_dim = 100
vocab_size = len(vocabulary)
embedding_matrix = np.random.uniform(-0.05, 0.05, (vocab_size, embedding_dim))

# Create a dictionary to store the GloVe embeddings
glove_embeddings = {}
glove_file = 'glove.6B.100d.txt'
with open(glove_file, 'r', encoding='utf8') as f:
    for line in f:
        values = line.split()
        glove_word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        # Normalize the loaded vectors to have unit norm
        vector /= np.linalg.norm(vector)
        glove_embeddings[glove_word] = vector

# Load GloVe embeddings into the embedding matrix where possible
for idx, word in enumerate(vocabulary):
    if word in glove_embeddings:
        embedding_matrix[idx] = glove_embeddings[word]

In [16]:
class SentimentRNN_OOV(nn.Module):
    def __init__(self, embedding_matrix):
        super(SentimentRNN_OOV, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True
        self.lstm = nn.LSTM(embedding_dim, hidden_size=256, batch_first=True, bidirectional=True, num_layers=2, dropout=0.5)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, (h_n, c_n) = self.lstm(embeds)

        # Concatenate the final hidden states from both directions
        out = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)

        out = self.dropout(out)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = torch.sigmoid(self.fc2(out))
        return out.squeeze()

In [17]:

# Instantiate the model
model = SentimentRNN_OOV(embedding_matrix)

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2, factor=0.5, verbose=True)

# Training loop with validation and early stopping
num_epochs = 30
patience = 5  # Early stopping patience
best_val_accuracy = 0
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)  # Gradient clipping
        optimizer.step()
        running_loss += loss.item() * sequences.size(0)
    epoch_loss = running_loss / len(train_dataset)

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for sequences, labels in val_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = model(sequences)
            predicted = (outputs >= 0.5).long()
            correct += (predicted == labels.long()).sum().item()
            total += labels.size(0)
    val_accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Learning rate scheduling
    scheduler.step(val_accuracy)

    # Check for improvement
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_no_improve = 0
        # Save the best model
        torch.save(model.state_dict(), 'best_model_oov.pt')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print('Early stopping!')
            break


Epoch [1/30], Loss: 0.6439, Validation Accuracy: 0.7458
Epoch [2/30], Loss: 0.4071, Validation Accuracy: 0.7533
Epoch [3/30], Loss: 0.2064, Validation Accuracy: 0.7411
Epoch [4/30], Loss: 0.1132, Validation Accuracy: 0.7514
Epoch [5/30], Loss: 0.0660, Validation Accuracy: 0.7477
Epoch [6/30], Loss: 0.0235, Validation Accuracy: 0.7439
Epoch [7/30], Loss: 0.0137, Validation Accuracy: 0.7458
Early stopping!


In [18]:
# Load the best model and evaluate on the test set
model.load_state_dict(torch.load('best_model_oov.pt'))
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for sequences, labels in test_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        outputs = model(sequences)
        predicted = (outputs >= 0.5).long()
        correct += (predicted == labels.long()).sum().item()
        total += labels.size(0)
test_accuracy = correct / total
print(f'Test Accuracy: {test_accuracy:.4f}')

<ipython-input-18-c4ada12bb9c3>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_oov.pt'))


Test Accuracy: 0.7730


In [19]:
# biLSTM Model
class SentimentBiLSTM(nn.Module):
    def __init__(self, embedding_matrix):
        super(SentimentBiLSTM, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True
        self.lstm = nn.LSTM(embedding_dim, hidden_size=256, batch_first=True, bidirectional=True, num_layers=2, dropout=0.5)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, (h_n, c_n) = self.lstm(embeds)

        # Concatenate the final hidden states from both directions
        out = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)

        out = self.dropout(out)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = torch.sigmoid(self.fc2(out))
        return out.squeeze()

In [20]:
# biLSTM Model
class SentimentBiLSTM(nn.Module):
    def __init__(self, embedding_matrix):
        super(SentimentBiLSTM, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True
        self.lstm = nn.LSTM(embedding_dim, hidden_size=256, batch_first=True, bidirectional=True, num_layers=2, dropout=0.5)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, (h_n, c_n) = self.lstm(embeds)

        # Concatenate the final hidden states from both directions
        out = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)

        out = self.dropout(out)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = torch.sigmoid(self.fc2(out))
        return out.squeeze()


# biGRU Model
class SentimentBiGRU(nn.Module):
    def __init__(self, embedding_matrix):
        super(SentimentBiGRU, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True
        self.gru = nn.GRU(embedding_dim, hidden_size=256, batch_first=True, bidirectional=True, num_layers=2, dropout=0.5)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        gru_out, h_n = self.gru(embeds)

        # Concatenate the final hidden states from both directions
        out = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)

        out = self.dropout(out)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = torch.sigmoid(self.fc2(out))
        return out.squeeze()




In [21]:
# Instantiate the models
bilstm_model = SentimentBiLSTM(embedding_matrix)
bigru_model = SentimentBiGRU(embedding_matrix)

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bilstm_model.to(device)
bigru_model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
bilstm_optimizer = torch.optim.Adam(bilstm_model.parameters(), lr=0.001, weight_decay=1e-5)
bigru_optimizer = torch.optim.Adam(bigru_model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(bilstm_optimizer, 'max', patience=2, factor=0.5, verbose=True)

In [22]:
# Training loop with validation and early stopping for biLSTM
num_epochs = 30
patience = 5  # Early stopping patience
best_val_accuracy = 0
epochs_no_improve = 0

for epoch in range(num_epochs):
    bilstm_model.train()
    running_loss = 0.0
    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        bilstm_optimizer.zero_grad()
        outputs = bilstm_model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(bilstm_model.parameters(), max_norm=1)  # Gradient clipping
        bilstm_optimizer.step()
        running_loss += loss.item() * sequences.size(0)
    epoch_loss = running_loss / len(train_dataset)

    # Validation
    bilstm_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for sequences, labels in val_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = bilstm_model(sequences)
            predicted = (outputs >= 0.5).long()
            correct += (predicted == labels.long()).sum().item()
            total += labels.size(0)
    val_accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Learning rate scheduling
    scheduler.step(val_accuracy)

    # Check for improvement
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_no_improve = 0
        # Save the best model
        torch.save(bilstm_model.state_dict(), 'best_bilstm_model.pt')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print('Early stopping!')
            break

# Load the best model and evaluate on the test set for biLSTM
bilstm_model.load_state_dict(torch.load('best_bilstm_model.pt'))
bilstm_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for sequences, labels in test_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        outputs = bilstm_model(sequences)
        predicted = (outputs >= 0.5).long()
        correct += (predicted == labels.long()).sum().item()
        total += labels.size(0)
bilstm_test_accuracy = correct / total
print(f'biLSTM Test Accuracy: {bilstm_test_accuracy:.4f}')

Epoch [1/30], Loss: 0.6693, Validation Accuracy: 0.7326
Epoch [2/30], Loss: 0.4406, Validation Accuracy: 0.7467
Epoch [3/30], Loss: 0.2294, Validation Accuracy: 0.7373
Epoch [4/30], Loss: 0.1194, Validation Accuracy: 0.7392
Epoch [5/30], Loss: 0.0693, Validation Accuracy: 0.7561
Epoch [6/30], Loss: 0.0394, Validation Accuracy: 0.7533
Epoch [7/30], Loss: 0.0186, Validation Accuracy: 0.7439
Epoch [8/30], Loss: 0.0210, Validation Accuracy: 0.7439
Epoch [9/30], Loss: 0.0077, Validation Accuracy: 0.7477
Epoch [10/30], Loss: 0.0017, Validation Accuracy: 0.7345
Early stopping!


<ipython-input-22-1d999e0fdf7e>:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  bilstm_model.load_state_dict(torch.load('best_bilstm_model.pt'))


biLSTM Test Accuracy: 0.7702


In [23]:
# Training loop with validation and early stopping for biGRU
num_epochs = 30
patience = 5  # Early stopping patience
best_val_accuracy = 0
epochs_no_improve = 0

for epoch in range(num_epochs):
    bigru_model.train()
    running_loss = 0.0
    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        bigru_optimizer.zero_grad()
        outputs = bigru_model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(bigru_model.parameters(), max_norm=1)  # Gradient clipping
        bigru_optimizer.step()
        running_loss += loss.item() * sequences.size(0)
    epoch_loss = running_loss / len(train_dataset)

    # Validation
    bigru_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for sequences, labels in val_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = bigru_model(sequences)
            predicted = (outputs >= 0.5).long()
            correct += (predicted == labels.long()).sum().item()
            total += labels.size(0)
    val_accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Learning rate scheduling
    scheduler.step(val_accuracy)

    # Check for improvement
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_no_improve = 0
        # Save the best model
        torch.save(bigru_model.state_dict(), 'best_bigru_model.pt')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print('Early stopping!')
            break

# Load the best model and evaluate on the test set for biGRU
bigru_model.load_state_dict(torch.load('best_bigru_model.pt'))
bigru_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for sequences, labels in test_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        outputs = bigru_model(sequences)
        predicted = (outputs >= 0.5).long()
        correct += (predicted == labels.long()).sum().item()
        total += labels.size(0)
bigru_test_accuracy = correct / total
print(f'biGRU Test Accuracy: {bigru_test_accuracy:.4f}')

Epoch [1/30], Loss: 0.6342, Validation Accuracy: 0.7608
Epoch [2/30], Loss: 0.3909, Validation Accuracy: 0.7664
Epoch [3/30], Loss: 0.1859, Validation Accuracy: 0.7495
Epoch [4/30], Loss: 0.0921, Validation Accuracy: 0.7430
Epoch [5/30], Loss: 0.0497, Validation Accuracy: 0.7467
Epoch [6/30], Loss: 0.0251, Validation Accuracy: 0.7373
Epoch [7/30], Loss: 0.0172, Validation Accuracy: 0.7251
Early stopping!


<ipython-input-23-9a625a24d354>:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  bigru_model.load_state_dict(torch.load('best_bigru_model.pt'))


biGRU Test Accuracy: 0.8011


In [24]:
# CNN Model
class SentimentCNN(nn.Module):
    def __init__(self, embedding_matrix):
        super(SentimentCNN, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True
        self.conv1 = nn.Conv1d(embedding_dim, 128, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x).permute(0, 2, 1)
        conv_out = self.conv1(embeds)
        pool_out = torch.max(conv_out, dim=-1)[0]
        out = self.dropout(pool_out)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = torch.sigmoid(self.fc2(out))
        return out.squeeze()

In [25]:
# Instantiate the model
cnn_model = SentimentCNN(embedding_matrix)

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cnn_model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
cnn_optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(cnn_optimizer, 'max', patience=2, factor=0.5, verbose=True)

# Training loop with validation and early stopping
num_epochs = 30
patience = 5  # Early stopping patience
best_val_accuracy = 0
epochs_no_improve = 0

for epoch in range(num_epochs):
    cnn_model.train()
    running_loss = 0.0
    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        cnn_optimizer.zero_grad()
        outputs = cnn_model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(cnn_model.parameters(), max_norm=1)  # Gradient clipping
        cnn_optimizer.step()
        running_loss += loss.item() * sequences.size(0)
    epoch_loss = running_loss / len(train_dataset)

    # Validation
    cnn_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for sequences, labels in val_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = cnn_model(sequences)
            predicted = (outputs >= 0.5).long()
            correct += (predicted == labels.long()).sum().item()
            total += labels.size(0)
    val_accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Learning rate scheduling
    scheduler.step(val_accuracy)

    # Check for improvement
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_no_improve = 0
        # Save the best model
        torch.save(cnn_model.state_dict(), 'best_cnn_model.pt')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print('Early stopping!')
            break

# Load the best model and evaluate on the test set
cnn_model.load_state_dict(torch.load('best_cnn_model.pt'))
cnn_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for sequences, labels in test_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        outputs = cnn_model(sequences)
        predicted = (outputs >= 0.5).long()
        correct += (predicted == labels.long()).sum().item()
        total += labels.size(0)
cnn_test_accuracy = correct / total
print(f'CNN Test Accuracy: {cnn_test_accuracy:.4f}')

Epoch [1/30], Loss: 0.6602, Validation Accuracy: 0.7129
Epoch [2/30], Loss: 0.4174, Validation Accuracy: 0.7702
Epoch [3/30], Loss: 0.1891, Validation Accuracy: 0.7692
Epoch [4/30], Loss: 0.0739, Validation Accuracy: 0.7589
Epoch [5/30], Loss: 0.0294, Validation Accuracy: 0.7617
Epoch [6/30], Loss: 0.0131, Validation Accuracy: 0.7598
Epoch [7/30], Loss: 0.0063, Validation Accuracy: 0.7645
Early stopping!


<ipython-input-25-b31d088b4292>:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cnn_model.load_state_dict(torch.load('best_cnn_model.pt'))


CNN Test Accuracy: 0.7814
